# Model Design: 如何选择好的模型结构

<font color=norange>**model框架：**
$$
input \rightarrow \underset{optimization}{{\underbrace{\overset{f(x,w)}{\overbrace{feature\ representation + classifier}} \rightarrow \overset{Div,reg}{ \overbrace{ cost function}}}}} 
$$</font>

**整个model可以拆分为两大类问题：模型结构设计和优化求解**

1. 设计一套好的模型结构，包括选择三件套$f(x,w), Div和Reg$，使min expected Loss能尽可能接近min Error这一真实目标
2. 选择好的优化方法(optimization method)，能以高效的方式找到相对好的local min

## I. 目标分解
### I.1 结合求解方法的目标转变
**target 1**: **expected error/test error**<font color=red>[真实目标]</font>:   $min\ E(error(f(x, w), y))$ \
<font color=green>$\Downarrow$ gap1: 让优化问题可以用GD求解</font> \
**target 2**: **expected divergence**<font color=red>[可微]</font>:   $min\ E(Div(f(x, w), y))$ \
<font color=green>$\Downarrow$ gap2: Loss(w)是E(Div)的无偏估计，问题不大</font> \
**target 3**: **test error**<font color=red>[可统计/抽样]</font>： $min\ Loss(w)\ =\ min\ \frac{1}{N}\sum_{i=1}^{N}Div(f(x,w),\ y)$ \
<font color=green>$\Downarrow$ gap3: **reg**增加了bias以减少Variance</font> \
**target 4**: **training error**：$min\ Loss(w)\ =\ min\ \frac{1}{N}\sum_{i=1}^{N}Div(f(x,w),\ y)\ +\ reg$

<font color=blue>注：上述变换过程不唯一。</font>如:perceptron将test error转为$min\ \frac{1}{N}\sum_{i=1}^{N}Error(f(x,w),\ y)$，但这种形式不可微，适用范围有限。

<font color=blue>注：**training目标中加上reg item的作用：**</font>用regularization控制overfit。 \
实现方式：reg可以降低training error中第一个part的variance，此时，training error与test error结果会更一致<font color=red>[同小同大，而不是值相等]</font>。所以，在training set上优化后，真正目的是降低的test error

### I.2 **Gap影响因素分析**

| error类型 | 公式 | error大小 | 影响因素 |
| --- | --- | --- | --- |
| truth | $E(error(f^*, y))$ , 其中$f^*=Ey$| 0 | - |
| exp error | E(error(f(x, w), y)) | A = Gap1 | f(·)表达力，w优化结果 |
| exp div | E(Div(f(x, w), y)) | B = A + Gap2 | div(·), y的表达方式 |
| test error | 1/N$\sum_{i=1}^{N}$Div(f(x,w),y) | C = B + Gap3 | 样本数量和多样性 |
| training error | 1/N$\sum_{i=1}^{N}$Div(f(x,w),y) + reg | D = C + Gap4 | 分布符合reg对应假设 |


#### I.2.1 <font color=blue>Gap1：取决于$f(·)$的表达力和w最终优化的结果</font>
model的representation power越大，并且w优化结果越好，则A越小。<font color=brown>[这里w的优化结果在下一部分opt method讨论]</font>

#### I.2.2 <font color=blue>Gap2：loss function的形式和y的形式决定</font>
1. 典型设定 \
(1) MSE直接选择了mean square \
(2) 常见的还有Likelihood, KL divergence, cross entropy \
(3) y的表达方式，比如分类问题中用{0, 1}还是{-1, 1}
2. min loss function与min expected error目标不一致的例子 \
(1) 模型设定：二分类问题，输入是一维数据x，用sigmoid function + cross entropy loss \
(2) 不一致问题：
<img src="pics/sigmoidfunction.png" alt="alt text" width="360">\
如图，sigmoid function的弯曲形态是固定的$f(x)=\frac{1}{1+e^{-x}}$不会改变，子啊不同的样本条件下，该曲线会左右移动来实现min loss以完成分类任务。\
假设A类样本都在0点右边，B类样本在0点左边，本来按照图中的原本形态是可以正确分类的。\
但是如果大量的A类型样本取值聚集在非常靠近0点的右侧附近，而B类样本只有少数处于0点左侧附近，那么min loss会将curve往左挤压，导致那些从左侧靠近0点的B类样本被分错。
3. 实践影响: <font color=green>divergence带来的问题在classifier中通常不是非常显著</font> \
**[补充说明]**<font color=blue>上例中，如果输入是Wx，也就是多变量，那么问题不大，因为可以通过w的优化找到合适的w，使得linear classifier输出的score不要挤压在0值附近。</font>

#### I.2.3 <font color=blue>Gap3：影响小</font>
由于experienced risk是expected divergence的无偏估计，对样本量的要求通常能够满足，所以Gap3问题最小

#### I.2.4 <font color=blue>Gap4：增加的reg项是以bias为代价降低varince</font>
<font color=green>Gap4在training data上就是reg item的值。reg的合理性需要weights的真实分布符合reg形态对应的假设。</font>
*如果分布符合reg对应的假设，比如使用L2 reg时，分布符合smooth假设*，那么min training error得到的w会让$f(x,w)$的variance更小，代价是$f(x, w)$的bias会增加。*注意variance和bias所指向的评价对象是$f(x,w)$* 

<font color=norange>**这里的bias和variance影响的是B，以及一定程度上A的大小，而非Gap4：**</font>
1. 假如training data的量很大，且其分布代表了真实分布。那么Gap3可以忽略，统计值和期望值可以视为相同值
2. 从期望值来看，假如是regression问题，那么Div取MSE合理，还可以忽略Gap2，此时，A = B，用$E(MSE)$表达第一目标expected error合理。
bias和variance可以准确表达expected error的大小。
3. 由于E(MSE)可以做分解，有trade-off式子：
$$E(Div) = E(MSE) = variance + bias + irreducible$$
可以通过降低variance和bias来降低E(MSE)，从而降低最终目标expected error。

## II. Bias-Variance Tradeoff和传统ML的观点
### II.1 目标：降低MSE
1. <font color=red>**基于MSE的bias-variance分解**</font> \
· 假设：$y = f^{*}(x) + ε$， 其中，ε是与x无关的因素，$E(ε)=0, Var(ε)=σ^{2}$ \
· 样本D的估计量为$\hat{f}_{D}(x, w)$，简写为$\hat{f}(x,w)$，或$\hat{f}$
$$
\begin{align}
E_{D}(\hat{f_{D}}(x,w)-y)^{2}
&=E_{D}[(\hat{f_{D}}(x, w)-f^{*}(x))+(f^{*}(x)-y)]^{2}\\
&=E_{D}(\hat{f_{D}}(x,w)-f^{*}(x))^{2} + Eε^{2}\\    
&=E_{D}(\hat{f_{D}}(x,w)-f^{*}(x))^{2} + σ_{ε}^{2}\\
&=E_{D}[(\hat{f_{D}}(x,w)-E_{D}\hat{f_{D}}(x,w))+(E_{D}\hat{f_{D}}(x,w)-f^{*}(x))]^{2}+ σ_{ε}^{2}\\
&=E_{D}[(\hat{f_{D}}-E_{D}\hat{f_{D}})+(E_{D}\hat{f_{D}}-f^{*})]^{2}+ σ_{ε}^{2}\\
&=E_{D}(\hat{f_{D}}-E_{D}\hat{f_{D}})^{2}+E_{D}(E_{D}\hat{f_{D}}-f^{*})^{2}+ σ_{ε}^{2}\\
&=sample\ Var + Bias\ of\ estimate + irreducible\ part
\end{align}
$$
<font color=red>[注：实验时要用test set来估这里的期望，D=test set。不能用training set]</font>

2. 理解variance和bias \
·<font color=blue> **variance:** </font>\
(1) 衡量当训练样本改变时，估计量会发生变化的程度。variance大意味着，样本发生一点点改变，估计结果就会发生很大的改变。\
(2) 通常flexibility越大的model有更大的variance。因为flexibility大的model，会有更多的拟合能力，在min loss的过程中，会拟合noise，而使得variance增大。\
·<font color=blue> **bias:** </font>\
(1) bias衡量的是模型本身与真实函数之间的差异。\
(2) 通常情况下，flexibility越大的模型，bias越小。因为flexibility越大，意味着模型的拟合能力越强，拟合真实函数的可能性就越大。

### II.2 传统ML在实验中观察到U形曲线，形成了bias variance trade-off的观点
1. 传统ML从实验中观察到：f(x,w)的flexibility与MSE的关系呈U形。在flexibility达到某一最优值后，如果继续增加自由度，反而会让MSE上升。
<img src="pics/biasvartradeoff.png" alt="alt text" width="500">

2. 传统ML对U形曲线现象的解释 \
<img src="pics/biasvartradeoff2.png" alt="alt text" width="500"> \
随着模型flexibility提升，bias下降variance上升。这两者下降和上升的相对关系决定了MSE的总体变化是增加还是减少。\
<font color=orange>按照传统ML在简单模型上观察到的经验，flexibility增加的时候，bias一开始减少的快，后面减少得越来越慢。而variance一开始增加得慢，随着flexibility增加会越来越大，这两总情况加总就会得到U形曲线。</font> \
<font color=red>**但是，这个现象在DL中却没有明显证据。**</font>

## III. DL结构下最小化MSE的思路
### III.1 DL之后的观点改变
<font color=blue>**增加flexibility可以减少bias，更大更深的网络可以实现极大的表达力。用大量的数据做拟合，再配合regularization方法减少variance，可以实现MSE的持续下降。**</font>

### III.2 Bias的处理方法：model selection/representation power，data augmentation
1. 提高model的表达力，用universal approximator
2. 增加数据量：data augmentation，data generation

### III.3 Variance的处理方法：data, model selection, regularization
<font color=norange>第1类variance来源：**由数据样本的随机性带来的**</font>
1. 整体数据集本身：**more data，training data和test data分布一致**
2. 数据集的使用方式，SGD导致fluctuation：\
(1) 要**shuffle** \
(2) SGD由于sample小，即使shuffle也会有很大的fluctuate，用loss curve呈现的波动会更大。用SGD时要额外处理variance的问题。<font color=green>[详见SGD的opt章节笔记]</font>

<font color=norange>第2类variance来源：**由model的flexibility带来的，会导致overfit**</font>
1. 选择Variance小的model结构和优化方法：比如Cross Entropy+GD比Perceptron算法的Variance更小，但bias更大。\
*说明：* \
Perceptron中增加一个outlier样本会给算法结果带来很大的改变，因此variance很大。但是它的bias很小。相反，用CE或者MLE+Gradient Descent的优化过程受单个样本的影响小，variance小，但是bias会大。<font color=red>**这里也可以看出，loss function和优化方法通常是整体考虑的。**</font>
2. 用**regularization**来限制weight的波动：[详见Regularization章节] \
(1) 比如：***weight decay(L2 Norm), weight clipping, dropout(主要用在FC层，CNN中FC被pooling替代后就不适用)*** \
(2) 在ML中，L2 norm与Bayesian等价，可以理解成Bayesian中的先验信息在这里转换成了L2 norm形式的Regularization约束。DL没有Bayesian形式，但仍然可以将L2 norm视为“先验约束”。\
(3) 在DL中，L2对应的“先验约束”意味着，接受“真实分布往往是smooth的”假设